In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Filename   : 2-TVLQR.ipynb
Author     : Vrachoriti Alexandra
Date       : 2026-02-04
Description: Implementation of the Backward Pass (Riccati Ricursion) of Time-Variant Linear Qudratic Regulator (TVLQR) 
"""

'\nFilename   : 2-TVLQR.ipynb\nAuthor     : Vrachoriti Alexandra\nDate       : 2026-02-04\nDescription: Implementation of the Backward Pass (Riccati Ricursion) of Time-Variant Linear Qudratic Regulator (TVLQR) \n'

In [2]:
# --- Cell 1 ---
%matplotlib inline
import numpy as np
import pandas as pd

import jax.numpy as jnp
from jax import jacfwd

from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="vpython")

In [3]:
# --- Cell 2 ---
# Initializations
Ns = 4               # dimension of state
Nc = 2               # dimension of control

tf = 5.              # total time
dt = 0.05            # simulation time step
S = round(tf/dt) + 1 # number of knot points

# Parameters
g = 9.81 # gravity (m/s^2)

# [SHOULDER, ELBOW] -> [link 1 (upper), link 2 (lower)]
# SHOULDER
l1 = 0.05                     # length (m)
m1 = 0.10548177618443695      # mass (kg)
com1 = 0.05                   # center of mass (m)
I1 = 0.00046166221821039165   # moment of inertia (kg * m^2)
coulomb_fric1 = 0.00305       # coulomb friction (N)
damp1 = 7.634058385430087e-12 # damping

# ELBOW
l2 = 0.05                     # length (m)
m2 = 0.07619744360415454      # mass (kg)
com2 = 0.03670036749567022    # center of mass (m)
I2 = 0.00023702395072092597   # moment of inertia (kg * m^2)
coulomb_fric2 = 0.0007777     # coulomb friction (N)
damp2 = 0.0005106535523065844 # damping

# Limits
torque_limit = 0.09
# torque_limit = 0.15

In [4]:
# --- Cell 3 ---
# Cost
Q = np.diag([500., 500., 10., 10.]) 
R = 100. * np.eye(Nc)
QF = 2000. * np.eye(Ns)

Let's get the results of trajectory optimization ($\bar{x}, \bar{u}$)!

In [5]:
# --- Cell 4 ---
results = pd.read_csv('results-tl-009.csv')
# results = pd.read_csv('results-tl-015.csv')
q1_bar = results['q1_bar'].values[::2]
q2_bar = results['q2_bar'].values[::2]
q1_dot_bar = results['q1_dot_bar'].values[::2]
q2_dot_bar = results['q2_dot_bar'].values[::2]
u1_bar = results['u1_bar'].values[::2]
u2_bar = results['u2_bar'].values[::2]

xb = np.column_stack((q1_bar, q2_bar, q1_dot_bar, q2_dot_bar))
xb = jnp.array(xb).T 

ub = np.column_stack((u1_bar, u2_bar))
ub = jnp.array(ub).T

In [6]:
# --- Cell 3 ---
def M(q):
    q1, q2 = q[0,0], q[1,0]
    A = jnp.array([[I1 + I2 + l1**2 * m2 + 2 * l1 * m2 * com2 * jnp.cos(q2), I2 + l1 * m2 * com2 * jnp.cos(q2)],
                  [                       I2 + l1 * m2 * com2 * jnp.cos(q2),                                I2]])

    return A

def C(q, q_dot):
    q1, q2 = q[0,0], q[1,0]
    q1_dot, q2_dot = q_dot[0,0], q_dot[1,0]
    A = jnp.array([[-2 * q2_dot * l1 * m2 * com2 * jnp.sin(q2), -q2_dot * l1 * m2 * com2 * jnp.sin(q2)],
                  [      q1_dot * l1 * m2 * com2 * jnp.sin(q2),                                      0]])
    return A

def G(q):
    q1, q2 = q[0,0], q[1,0]
    A = jnp.array([[-g * m1 * com1 * jnp.sin(q1) - g * m2 * (l1 * jnp.sin(q1) + com2 * jnp.sin(q1+q2))],
                  [                          -g * m2 * com2 * jnp.sin(q1+q2)                          ]])
    return A

def F(q_dot):
    q1_dot, q2_dot = q_dot[0,0], q_dot[1,0]
    
    A = jnp.array([[damp1 * q1_dot + coulomb_fric1 * jnp.atan(100 * q1_dot)],
                   [damp2 * q2_dot + coulomb_fric2 * jnp.atan(100 * q2_dot)]])
    return A

In [7]:
# --- Cell 6 ---
def dynamics(x, u):
    q     = jnp.block([[x[0,0], x[1,0]]]).T
    q_dot = jnp.block([[x[2,0], x[3,0]]]).T

    lhs = M(q)
    rhs = u - C(q,q_dot) @ q_dot + G(q) # - F(q_dot)
    q_ddot = jnp.linalg.solve(lhs, rhs)

    return jnp.block([[q_dot], [q_ddot]])

In [8]:
# --- Cell 7 ---
# Runge-Kutta 4th Order integration
def rk4(x, u):
    f1 = dynamics(x, u)
    f2 = dynamics(x + f1 * dt/2, u)
    f3 = dynamics(x + f2 * dt/2, u)
    f4 = dynamics(x + f3 * dt, u)
    
    return x + dt/6 * (f1 + 2*f2 + 2*f3 + f4)

# Linearization
def linearize(x_bar, u_bar):
    A = (jacfwd(rk4, argnums=0)(x_bar, u_bar)).reshape(Ns,Ns)
    B = (jacfwd(rk4, argnums=1)(x_bar, u_bar)).reshape(Ns,Nc)
    
    return A, B

In [9]:
# --- Cell 8 ---
# Cost function
def cost(xs, us):
    c = 0.5 * xs[-1].T @ QF @ xs[-1]
    for k in range(S-1):
        c += 0.5 * xs[k].T @ Q @ xs[k]
        c += 0.5 * us[k].T @ R @ us[k]
    return c[0][0]

# **3. Controlling via TVLQR**

1. Write a function in Python called $\texttt{my\_controller()}$ that takes as input the current state
of the system, and outputs the controls to be applied to the system. You should linearize
around the trajectory that the previous step outputs.

In [10]:
# --- Cell 9 ---
def my_controller(current_x):
    # Initializations
    Ks = [jnp.zeros((Nc, Ns))] * (S-1)
    Ps = [jnp.zeros((Ns, Ns))] * S

    # 1. Backward Pass: Riccati Recursion
    Ps[S-1] = QF
    for k in range(S-2, -1, -1):
        x_bar = xb[:Ns, k].reshape(Ns,-1)
        u_bar = ub[:Nc, k].reshape(Nc,-1)
        
        A, B = linearize(x_bar, u_bar)
        lhs = (R + B.T @ Ps[k+1] @ B)
        rhs = B.T @ Ps[k+1] @ A
        Ks[k] = jnp.linalg.solve(lhs, rhs)
        Ps[k] = Q + A.T @ Ps[k+1] @ (A - B @ Ks[k])

    return Ks

In [11]:
# --- Cell 10 ---
# Initial state
x0 = jnp.array([[0., 0., 0., 0.]]).T

# Results
Ks = my_controller(x0)

In [12]:
# --- Cell 11 ---
Ks = [np.array(x) for x in Ks]
Ks = np.array(Ks)
Ks = Ks.reshape(Ks.shape[0], -1)

df = pd.DataFrame(Ks, columns=[f'col{i}' for i in range(Ks.shape[1])])
df.to_csv('gains-tl-009.csv', index=False)
# df.to_csv('gains-tl-015.csv', index=False)